# IHLT final project - Semantic Textual Similarity
Nikita Belooussov and Santiago del Rey Juárez

## Introduction

In [ ]:
import csv
import os
import pandas as pd
import numpy as np
import nltk
from nltk.metrics import jaccard_distance
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk.stem import WordNetLemmatizer
import string
import spacy
from nltk.metrics import jaccard_distance

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('conll2000')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet_ic')
nltk.download('wordnet')

nlp = spacy.load("en_core_web_sm")


brown_ic=wordnet_ic.ic('ic-brown.dat')

!pip install svgling

In [5]:
def read_file(file_path):
    return pd.read_csv(file_path, sep='\t', lineterminator='\n', names=['sentence0', 'sentence1'], header=None,
                       quoting=csv.QUOTE_NONE)

Lemmatize

In [ ]:
def lemmatize(p):
    if p[1][0] in {'N','V','J','R'}: #N- noun, V- verb, J- adjective, R-adverb
        if p[1][0] is 'J':  #this is used due to wordnet using a different label for adjectives than one given by nltk
            return wnl.lemmatize(p[0].lower(), pos=wordnet.ADJ)
        return wnl.lemmatize(p[0].lower(), pos=p[1][0].lower())
    return p[0]

Computes the Jaccard Distance 

In [ ]:
def remove_punct_and_lower(s0, s1):
    s0 = [w.lower() for w in s0 if not all(c in punct for c in w)]
    s1 = [w.lower() for w in s1 if not all(c in punct for c in w)]
    return s0, s1

def compute_jaccard(sentence_0, sentence_1, type = 'nltk'):
    s0 = nltk.word_tokenize(sentence_0)
    s1 = nltk.word_tokenize(sentence_1)

    if type == 'lemma':
        pairs_s0 = nltk.pos_tag(s0)
        pairs_s1 = nltk.pos_tag(s1)
        s0 = [lemmatize(pair) for pair in pairs_s0]
        s1 = [lemmatize(pair) for pair in pairs_s1]
    
    if type == 'punct':
        s0, s1 = remove_punct_and_lower(s0, s1)
    return 1 - jaccard_distance(set(s0), set(s1))


    if type == "NE":
        doc = nlp(sentence_0.lower())
        
        with doc.retokenize() as retokenizer:
            tokens = [token for token in doc]
            for ent in doc.ents:
                retokenizer.merge(doc[ent.start:ent.end], 
                          attrs={"LEMMA": " ".join([tokens[i].text for i in range(ent.start, ent.end)])})
        s0_ne = [token.text for token in doc]
    
        doc = nlp(sentence_1.lower())
    
        with doc.retokenize() as retokenizer:
            tokens = [token for token in doc]
            for ent in doc.ents:
                retokenizer.merge(doc[ent.start:ent.end], 
                          attrs={"LEMMA": " ".join([tokens[i].text for i in range(ent.start, ent.end)])})

        s1_ne = [token.text for token in doc]

        #print('Sentence 0:')
        #print(s0_ne)
        #print('Sentence 1:')
        #print(s1_ne)

        return 1 - jaccard_distance(set(s0_ne), set(s1_ne))
    
    if type=="Leacock" or type=="Lin":
        syn=[]
        for word in words:
            try:
                syn.append(wn.synsets(word[0],word[1].lower()[0])[0])
            except:
                pass
        
        if type=="Leacock":
            lSimA=round(syn[0].lch_similarity(syn[0]),3)
            try:
                lSim=round(word.lch_similarity(word2)/lSimA,3)
            except:
                lSimA.append(0)
        if type=="Lin":
            liSimA=round(syn[0].lin_similarity(syn[0],brown_ic),3)
            try:
                liSim=round(word.lin_similarity(word2,brown_ic)/liSimA,3)
            except:
                liSimA.append(0)

In [8]:
path = os.path.join('data', 'train', 'STS.input.MSRpar.txt')



df_input = read_file(path)
df_input.head()

,sentence0,sentence1
0,But other sources close to the sale said Viven...,But other sources close to the sale said Viven...
1,Micron has declared its first quarterly profit...,Micron's numbers also marked the first quarter...
2,The fines are part of failed Republican effort...,"Perry said he backs the Senate's efforts, incl..."
3,"The American Anglican Council, which represent...","The American Anglican Council, which represent..."
4,The tech-loaded Nasdaq composite rose 20.96 po...,The technology-laced Nasdaq Composite Index <....
